<a href="https://colab.research.google.com/github/thivagarradhakrishnan/Sentimental_Analysis/blob/main/SentimentalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flair
!pip install pyngrok
!pip install flask_ngrok
!ngrok authtoken 24KbxKL4nSj7XKyi6kHAeeOQUbz_6fpMhuTxVTGtsCbG4Dv4b

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import pandas as pd
import json
import os
import sys
import time
import requests
import flair
from flair.data import Sentence
import re

def recognize(source_url):
  # pandas dataframe display configuration
  pd.set_option('display.max_rows', 500)
  pd.set_option('display.max_columns', 500)
  pd.set_option('display.width', 1000)

  USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
  # csv file name
  FILE_NAME = 'ytb_comments.csv'

  # set parameters
  # filter comments by popularity or recent, 0:False, 1:True
  SORT_BY_POPULAR = 0
  # default recent False, change to 1 to download latest comments
  SORT_BY_RECENT = 0
  # set comment limit
  COMMENT_LIMIT = 50

  YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
  YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'

  def regex_search(text, pattern, group=1, default=None):
      match = re.search(pattern, text)
      return match.group(group) if match else default


  def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
      url = 'https://www.youtube.com' + endpoint['commandMetadata']['webCommandMetadata']['apiUrl']

      data = {'context': ytcfg['INNERTUBE_CONTEXT'],
              'continuation': endpoint['continuationCommand']['token']}

      for _ in range(retries):
          response = session.post(url, params={'key': ytcfg['INNERTUBE_API_KEY']}, json=data)
          if response.status_code == 200:
              return response.json()
          if response.status_code in [403, 413]:
              return {}
          else:
              time.sleep(sleep)

  def download_comments(YOUTUBE_VIDEO_URL, sort_by=SORT_BY_RECENT, language=None, sleep=0.1):
      session = requests.Session()
      session.headers['User-Agent'] = USER_AGENT
      response = session.get(YOUTUBE_VIDEO_URL)

      if 'uxe=' in response.request.url:
          session.cookies.set('CONSENT', 'YES+cb', domain='.youtube.com')
          response = session.get(YOUTUBE_VIDEO_URL)

      html = response.text
      ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
      if not ytcfg:
          return # Unable to extract configuration
      if language:
          ytcfg['INNERTUBE_CONTEXT']['client']['hl'] = language

      data = json.loads(regex_search(html, YT_INITIAL_DATA_RE, default=''))

      section = next(search_dict(data, 'itemSectionRenderer'), None)
      renderer = next(search_dict(section, 'continuationItemRenderer'), None) if section else None
      if not renderer:
          # Comments disabled?
          return

      needs_sorting = sort_by != SORT_BY_POPULAR
      continuations = [renderer['continuationEndpoint']]
      while continuations:
          continuation = continuations.pop()
          response = ajax_request(session, continuation, ytcfg)

          if not response:
              break
          if list(search_dict(response, 'externalErrorMessage')):
              raise RuntimeError('Error returned from server: ' + next(search_dict(response, 'externalErrorMessage')))

          if needs_sorting:
              sort_menu = next(search_dict(response, 'sortFilterSubMenuRenderer'), {}).get('subMenuItems', [])
              if sort_by < len(sort_menu):
                  continuations = [sort_menu[sort_by]['serviceEndpoint']]
                  needs_sorting = False
                  continue
              raise RuntimeError('Failed to set sorting')

          actions = list(search_dict(response, 'reloadContinuationItemsCommand')) + \
                    list(search_dict(response, 'appendContinuationItemsAction'))
          for action in actions:
              for item in action.get('continuationItems', []):
                  if action['targetId'] == 'comments-section':
                      # Process continuations for comments and replies.
                      continuations[:0] = [ep for ep in search_dict(item, 'continuationEndpoint')]
                  if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                      # Process the 'Show more replies' button
                      continuations.append(next(search_dict(item, 'buttonRenderer'))['command'])

          for comment in reversed(list(search_dict(response, 'commentRenderer'))):
              yield {'videoid': comment['commentId'],
                    'comment_text': ''.join([c['text'] for c in comment['contentText'].get('runs', [])]),
                    'likes': next(search_dict(comment, 'isHearted'), False)}

          time.sleep(sleep)

  def search_dict(partial, search_key):
      stack = [partial]
      while stack:
          current_item = stack.pop()
          if isinstance(current_item, dict):
              for key, value in current_item.items():
                  if key == search_key:
                      yield value
                  else:
                      stack.append(value)
          elif isinstance(current_item, list):
              for value in current_item:
                  stack.append(value)

  def main(url):
      """
      This function will save the comments into a dataframe and output a csv file
      By default, it will append the comments in csv, not overwriting them, change it in line 34
      To preview the comments in json, uncomment the lines - 22 and 23
      """
      df_comment = pd.DataFrame()
      try:
          youtube_url = url
          limit = COMMENT_LIMIT

          print('Downloading Youtube comments for video:', youtube_url)

          count = 0

          start_time = time.time()

          for comment in download_comments(youtube_url):

              df_comment = df_comment.append(comment, ignore_index=True)

              count += 1

              if limit and count >= limit:
                  break

          print("DataFrame Shape: ",df_comment.shape,"\nComment DataFrame: ")
          display(df_comment)

          if not os.path.isfile(FILE_NAME):
              df_comment.to_csv(FILE_NAME, encoding='utf-8', index=False)
          else:  # else it exists so append without writing the header
              df_comment.to_csv(FILE_NAME, mode='w', encoding='utf-8', index=False, header=True)

          print('\n[{:.2f} seconds] Done!'.format(time.time() - start_time))

      except Exception as e:
          print('Error:', str(e))
          sys.exit(1)

  youtube_URL = source_url
  main(youtube_URL)

  df=pd.read_csv('./ytb_comments.csv')

  pd.options.display.max_rows = None

  sentiment_model = flair.models.TextClassifier.load('en-sentiment')

  sentiment =[]
  confidence =[]
  for sentence in df['comment_text']:
      if sentence.strip() =="":
          sentiment.append("")
          confidence.append("")
      else:
          sample = flair.data.Sentence(sentence)
          sentiment_model. predict(sample)
          sentiment. append(sample. labels [0].value)
          confidence. append(sample. labels[0].score)

  df['sentiment']=sentiment
  df['confident']=confidence

  counts=df.sentiment.value_counts();
  df_result=pd.DataFrame(counts)
  df_result = df_result.reset_index()
  df_result.columns = ['Sentiment', 'Value']
  xArray=df_result['Sentiment'].to_numpy()
  yArray=df_result['Value'].to_numpy()
  x='['+','.join('"'+e+'"' for e in xArray)+']'
  y='['+','.join([str(e) for e in yArray])+']'
  data='['+x+','+y+']'
  return data

In [ ]:
from flask import *
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def home():
    return render_template('index.html')

@app.route("/sendurl/",methods=["post"])
def get_url():
    url=request.form['url']
    chart=recognize(url)
    return render_template('index.html',chart=chart)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://890a-34-86-44-100.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [15/May/2023 09:37:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/May/2023 09:37:56] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/May/2023 09:38:09] "POST /sendurl HTTP/1.1" 308 -
INFO:werkzeug:127.0.0.1 - - [15/May/2023 09:44:05] "POST /sendurl HTTP/1.1" 308 -
INFO:werkzeug:127.0.0.1 - - [15/May/2023 09:44:18] "POST /sendurl HTTP/1.1" 308 -


DataFrame Shape:  (20, 3) 
Comment DataFrame: 


<ipython-input-2-1b352c4a9873>:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_comment = df_comment.append(comment, ignore_index=True)
<ipython-input-2-1b352c4a9873>:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_comment = df_comment.append(comment, ignore_index=True)
<ipython-input-2-1b352c4a9873>:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_comment = df_comment.append(comment, ignore_index=True)
<ipython-input-2-1b352c4a9873>:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_comment = df_comment.append(comment, ignore_index=True)
<ipython-input-2-1b352c4a9873>:146: FutureWarning: The frame.append method is de

,videoid,comment_text,likes
0,UgwifTCbEDz8JA2npcZ4AaABAg,taylor is one of these few singers who have so...,False
1,UgzH0soUFeocvlsfooR4AaABAg,Cada vez amo más esta canción !,False
2,Ugwktz1VMHk_D_3X74l4AaABAg,"Essa música é simplesmente perfeita, faz meu c...",False
3,UgySBcErCBpSmK45Ukl4AaABAg,HER MATURE VOICE IS A PURE HEAVEN TO HEAR!!! ♥️,False
4,UgxgwWUtuvJHKLUIQNF4AaABAg,Amo tanto essa música ela me da borboletas no ...,False
5,UgxTIqw17T1hylBlxrx4AaABAg,Uma das primeiras músicas que eu ouvi dela kkk...,False
6,Ugz-DMEl7P1wNy3uhuB4AaABAg,the additional beat coming into play starting ...,False
7,UgxnyKMhczwJO3Edd-V4AaABAg,Essa música é perfeita❤❤ umas das minhas favs ...,False
8,Ugw19z1SMwt2cHVpa0F4AaABAg,no sabiamos que la necesitabamos hasta que la ...,False
9,UgzQuN_39j_YPINZKKx4AaABAg,Loved it❤,False



[0.87 seconds] Done!


INFO:werkzeug:127.0.0.1 - - [15/May/2023 09:44:28] "POST /sendurl/ HTTP/1.1" 200 -
